In [1]:
#Import packages
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import os
import sys
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib
from tensorflow.python.framework import ops
ops.reset_default_graph()

C:\Users\user\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Start a graph session
sess = tf.Session()

# Set model parameters
with tf.name_scope("parameters"):
    batch_size = 128
    data_dir = 'temp'
    output_every = 50
    epochs = 200
    eval_every = 500
    image_height = 32
    image_width = 32
    crop_height = 24
    crop_width = 24
    num_channels = 3
    num_targets = 10
    extract_folder = 'cifar-10-batches-bin'
    
    # Exponential Learning Rate Decay Params
    learning_rate = 0.1
    lr_decay = 0.1
    num_gens_to_wait = 250.

    # Extract model parameters
    image_vec_length = image_height * image_width * num_channels
    record_length = 1 + image_vec_length # ( + 1 for the 0-9 label)
logs_path = '/tmp2/tensorflow_logs/example/'

In [3]:
# Load data
data_dir = 'temp'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
cifar10_url = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

# Check if file exists, otherwise download it
data_file = os.path.join(data_dir, 'cifar-10-binary.tar.gz')
if os.path.isfile(data_file):
    pass
else:
    # Download file
    def progress(block_num, block_size, total_size):
        progress_info = [cifar10_url, float(block_num * block_size) / float(total_size) * 100.0]
        print('\r Downloading {} - {:.2f}%'.format(*progress_info), end="")
    filepath, _ = urllib.request.urlretrieve(cifar10_url, data_file, progress)
    # Extract file
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

In [4]:
# Define CIFAR reader
def read_cifar_files(filename_queue, distort_images = True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length)
    key, record_string = reader.read(filename_queue)
    record_bytes = tf.decode_raw(record_string, tf.uint8)
    image_label = tf.cast(tf.slice(record_bytes, [0], [1]), tf.int32)
  
    # Extract image
    image_extracted = tf.reshape(tf.slice(record_bytes, [1], [image_vec_length]),
                                 [num_channels, image_height, image_width])
    
    # Reshape image
    image_uint8image = tf.transpose(image_extracted, [1, 2, 0])
    reshaped_image = tf.cast(image_uint8image, tf.float32)
    # Randomly Crop image
    final_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, crop_width, crop_height)
    
    if distort_images:
        # Randomly flip the image horizontally, change the brightness and contrast
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image,max_delta=63)
        final_image = tf.image.random_contrast(final_image,lower=0.2, upper=1.8)

    # Normalize whitening
    final_image = tf.image.per_image_standardization(final_image)
    return final_image, image_label

In [5]:
# Create a CIFAR image pipeline from reader
def input_pipeline(batch_size, train_logical=True):
    if train_logical:
        files = [os.path.join(data_dir, extract_folder, 'data_batch_{}.bin'.format(i)) for i in range(1,6)]
    else:
        files = [os.path.join(data_dir, extract_folder, 'test_batch.bin')]
    filename_queue = tf.train.string_input_producer(files)
    image, label = read_cifar_files(filename_queue)
    
    # min_after_dequeue defines how big a buffer we will randomly sample
    #   from -- bigger means better shuffling but slower start up and more
    #   memory used.
    # capacity must be larger than min_after_dequeue and the amount larger
    #   determines the maximum we will prefetch.  Recommendation:
    #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 5000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch([image, label],
                                                        batch_size=batch_size,
                                                        capacity=capacity,
                                                        min_after_dequeue=min_after_dequeue)

    return example_batch, label_batch

In [6]:
# Define the model architecture, this will return logits from images
def cifar_cnn_model(x, batch_size, train_logical=True):
    
    # 입력 이미지 
    x_image = x 
    
    # 첫번째 convolutional layer : 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)합니다.
    with tf.variable_scope('layer1') as scope:
        W_conv1 = tf.get_variable("W", shape=[5, 5, 3, 64],initializer=tf.contrib.layers.xavier_initializer())
        b_conv1 = tf.Variable(tf.random_normal([64]))
        h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides =[1, 1, 1, 1], padding= 'SAME') + b_conv1)
        
        # 첫번째 Pooling layer : Max Pooling
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
        
    
    #두번째 convolutional layer : 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다. 
    with tf.variable_scope('layer2') as scope:
        W_conv2 = tf.get_variable("W", shape=[5, 5, 64, 64], initializer=tf.contrib.layers.xavier_initializer()) # Conv kernel is 5x5, across all prior 64 features and we create 64 more features 
        b_conv2 = tf.Variable(tf.random_normal([64]))
        h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2) # Convolve filter across prior output with stride size of 1, then # ReLU element wise
        
        # 두번째 pooling layer : Max Pooling
        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
        
    # Reshape output into a single matrix for multiplication for the fully connected layers
    reshaped_output = tf.reshape(h_pool2, [batch_size, -1])
    reshaped_dim = reshaped_output.get_shape()[1].value
    
    # Fully Connected Layer : 1 - 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 됩니다. 
    # 이를 384개의 특징들로 맵핑(maping)합니다. 
    with tf.variable_scope('layer_full') as scope:
        W_fc1 = tf.get_variable("W", shape=[reshaped_dim, 384], initializer=tf.contrib.layers.xavier_initializer()) # Conv kernel is 5x5, across all prior 64 features and we create 64 more features 
        b_fc1 = tf.Variable(tf.random_normal([384]))
        
        # Reshape output into a single matrix for multiplication for the fully connected layers
        h_fc1 = tf.nn.relu(tf.matmul(reshaped_output, W_fc1) + b_fc1) 

        
    with tf.variable_scope('layer_full2') as scope:
        # Dropout - 모델의 복잡도를 컨트롤합니다. 특징들의 co-adaptation을 방지합니다. 
        h_fc1_drop = tf.nn.dropout(h_fc1, 0.8)
        
        # Fully Connected Layer 2 - 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)합니다. 
        W_fc2 = tf.get_variable("W", shape=[384, 10], initializer=tf.contrib.layers.xavier_initializer()) 
        b_fc2 = tf.Variable(tf.random_normal([10]))
        
        logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2 
        y_pred = tf.nn.softmax(logits)

    return y_pred

In [7]:
# Loss function
def cifar_loss(logits, targets):
    
    # Get rid of extra dimensions and cast targets into integers
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    # Calculate cross entropy from logits and targets
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    # Take the average loss across batch size
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    
    tf.summary.scalar("loss", cross_entropy_mean)
    
    return cross_entropy_mean

In [8]:
# Train step
def train_step(loss_value, generation_num):
    
    # Our learning rate is an exponential decay after we wait a fair number of epochs
    model_learning_rate = tf.train.exponential_decay(learning_rate, generation_num,
                                                     num_gens_to_wait, lr_decay, staircase=True)
    # Create optimizer
    my_optimizer = tf.train.GradientDescentOptimizer(model_learning_rate)
    # Initialize train step
    train_step = my_optimizer.minimize(loss_value)
    
    return train_step

In [9]:
# Accuracy function
def accuracy_of_batch(logits, targets):
    
    # Make sure targets are integers and drop extra dimensions
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    # Get predicted values by finding which logit is the greatest
    batch_predictions = tf.cast(tf.argmax(logits, 1), tf.int32)
    # Check if they are equal across the batch
    predicted_correctly = tf.equal(batch_predictions, targets)
    # Average the 1's and 0's (True's and False's) across the batch size
    accuracy = tf.reduce_mean(tf.cast(predicted_correctly, tf.float32))

    tf.summary.scalar("accuracy", accuracy) 
    
    return accuracy

In [10]:
# Get data
print('Getting/Transforming Data.')
# Initialize the data pipeline
images, targets = input_pipeline(batch_size, train_logical=True)
# Get batch test images and targets from pipline
test_images, test_targets = input_pipeline(batch_size, train_logical=False)

Getting/Transforming Data.


In [11]:
# Declare Model
print('Creating the CIFAR10 Model.')
with tf.variable_scope('model_definition') as scope:
    # Declare the training network model
    model_output = cifar_cnn_model(images, batch_size)
    # This is very important!!!  We must set the scope to REUSE the variables,
    #  otherwise, when we set the test network model, it will create new random
    #  variables.  Otherwise we get random evaluations on the test batches.
    scope.reuse_variables()
    test_output = cifar_cnn_model(test_images, batch_size)
print('Done.')

Creating the CIFAR10 Model.
Done.


In [12]:
# Declare loss function
print('Declare Loss Function.')
loss = cifar_loss(model_output, targets)

# Create accuracy function
accuracy = accuracy_of_batch(test_output, test_targets)

# Summary
summary = tf.summary.merge_all()

# Create summary writer
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
global_step = 0

Declare Loss Function.


In [13]:
# Create training operations
print('Creating the Training Operation.')
generation_num = tf.Variable(0, trainable=False)
train_op = train_step(loss, generation_num)

# Initialize Variables
print('Initializing the Variables.')
init = tf.global_variables_initializer()
sess.run(init)

Creating the Training Operation.
Initializing the Variables.


In [14]:
# Initialize queue (This queue will feed into the model, so no placeholders necessary)
tf.train.start_queue_runners(sess=sess)

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 4836)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 13268)>,
 <Thread(QueueRunnerThread-input_producer_1-input_producer_1/input_producer_1_EnqueueMany, started daemon 3828)>,
 <Thread(QueueRunnerThread-shuffle_batch_1/random_shuffle_queue-shuffle_batch_1/random_shuffle_queue_enqueue, started daemon 11228)>]

In [15]:
# Train CIFAR Model
print('Starting Training')
train_loss = []
test_accuracy = []
for i in range(epochs):
    s, _, loss_value = sess.run([summary,train_op, loss])
    writer.add_summary(s, global_step=global_step)
    global_step += 1
    
    if (i+1) % output_every == 0:
        train_loss.append(loss_value)
        output = '{} epochs : Loss = {:.5f}'.format((i+1), loss_value)
        print(output)
    
    if (i+1) % eval_every == 0:
        [temp_accuracy] = sess.run([accuracy])
        test_accuracy.append(temp_accuracy)
        acc_output = ' --- Test Accuracy = {:.2f}%.'.format(100.*temp_accuracy)
        print(acc_output)
        
print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs")
        

Starting Training
50 epochs : Loss = 2.31892
100 epochs : Loss = 2.19319
150 epochs : Loss = 2.19093
200 epochs : Loss = 2.14684
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs
